import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

df_results = pd.read_csv(os.path.join(os.getcwd(), 'e_from_nms_conf.csv'))
df_results

In [ ]:
nms_group = df_results[df_results["nms"] >= 0.4].groupby("nms")
plt.plot(nms_group.groups.keys(), nms_group.mean(numeric_only = True).iloc[:, -3:], label=["YuNet", "MediaPipe", "Yolo-face"])
plt.legend()
plt.show()

conf_group = df_results[df_results["conf"] >= 0.4].groupby("conf")
plt.plot(conf_group.groups.keys(), conf_group.mean(numeric_only = True).iloc[:, -3:], label=["YuNet", "MediaPipe", "Yolo-face"])
plt.legend()
plt.show()

plt.scatter(df_results["nms"], df_results.iloc[:, -6], label="Yolo-face")
plt.scatter(df_results["nms"], df_results.iloc[:, -5], label="MediaPipe")
plt.scatter(df_results["nms"], df_results.iloc[:, -4], label="Yolo-face")
plt.legend()
plt.show()

plt.scatter(df_results["conf"], df_results.iloc[:, -6], label="Yolo-face")
plt.scatter(df_results["conf"], df_results.iloc[:, -5], label="MediaPipe")
plt.scatter(df_results["conf"], df_results.iloc[:, -4], label="Yolo-face")
plt.legend()
plt.show()

plt.plot(nms_group.groups.keys(), nms_group.apply((lambda x: (x.iloc[:, -3:] >= 3).sum())), label=["YuNet", "MediaPipe", "Yolo-face"])
plt.legend()
plt.show()

plt.plot(conf_group.groups.keys(), conf_group.apply((lambda x: (x.iloc[:, -3:] >= 3).sum())), label=["YuNet", "MediaPipe", "Yolo-face"])
plt.legend()
plt.show()

___

In [ ]:
import sys
sys.path.append("..") 

In [2]:
%load_ext autoreload
%autoreload 2

import itertools
import random
import os
import time
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import csv
import cv2
import torchvision.transforms as transforms

#libraries for yolo
from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression
from pytorchyolo.utils.loss import compute_loss

ModuleNotFoundError: No module named 'pytorchyolo'

In [5]:
REF_SET_celeba = pd.Series(["../images/img_celeba_100/" + i for i in list(os.walk('../images/img_celeba_100'))[0][2]]).sample(16)
REF_SET_wider = pd.Series(['../images/1--Handshaking/' + i for i in list(os.walk('../images/1--Handshaking'))[0][2]]).sample(16)
sample_set = pd.concat([REF_SET_celeba, REF_SET_wider]).reset_index(drop=True)

IndexError: list index out of range

In [4]:
sample_set

0                   ../images/img_celeba_100/099318.jpg
1                   ../images/img_celeba_100/099888.jpg
2                   ../images/img_celeba_100/099122.jpg
3                   ../images/img_celeba_100/099288.jpg
4                   ../images/img_celeba_100/099809.jpg
5                   ../images/img_celeba_100/099731.jpg
6                   ../images/img_celeba_100/099159.jpg
7                   ../images/img_celeba_100/099356.jpg
8                   ../images/img_celeba_100/099518.jpg
9                   ../images/img_celeba_100/099392.jpg
10                  ../images/img_celeba_100/099166.jpg
11                  ../images/img_celeba_100/099496.jpg
12                  ../images/img_celeba_100/099327.jpg
13                  ../images/img_celeba_100/099764.jpg
14                  ../images/img_celeba_100/099044.jpg
15                  ../images/img_celeba_100/099326.jpg
16    ../images/1--Handshaking/1_Handshaking_Handsha...
17    ../images/1--Handshaking/1_Handshaking_Han

In [ ]:
from scripts.utils import detach_cpu, tensor_to_np_img, save_tensor_img, clone_detach, open_img_as_tensor, display_img

In [ ]:
print("CUDA Available: ", torch.cuda.is_available())
main_yf = YoloFace()
device, model = main_yf.device, main_yf.yf_face_detector


# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

In [ ]:
def load_mask(filename, face_num, target_bbox):
    # Get the mask name and open it
    filename = "restored_mask_" + os.path.splitext(filename)[0] + "_" + str(face_num) + "_image_final.png"
    mask = cv2.imread(os.path.join('../', RESTORED_MASK_PATH, filename), 0)
    
    # Find the corresponding row of the mask in the FACES_DF to get the padding
    face_row = FACES_DF.loc[FACES_DF['filename'] == filename]
    padded_dim = (int(face_row["x2_pad"] - face_row["x1_pad"]), int(face_row["y2_pad"] - face_row["y1_pad"]))
    
    # Get the target dimensions based on the target bounding boxes
    target_dim = (int(target_bbox[2] - target_bbox[0]), int(target_bbox[3] - target_bbox[1]))
    
    # Get the number of white pixels in the mask
    num_white = dict(zip(*np.unique(mask, return_counts = True)))[255]
    # Return an clear if the number of white pixels is less than 10% of the target dimensions
    if num_white < int(target_dim[0] * target_dim[1] * 0.1):
        return torch.ones((1, 3, target_dim[1], target_dim[0])), False
    
    # Appply morphological transformation to the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (int(mask.shape[0] * 0.5), int(mask.shape[1] * 0.5)))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    mask = transforms.Compose([DEFAULT_TRANSFORMS])((mask, np.zeros((1, 5))))[0].unsqueeze(0)
    
    # Masks are always square and should be smaller than the padded dimensions and target dimensions
    current_dim = max(mask.shape)
    diff_x, diff_y = abs(padded_dim[0] - current_dim) / 2, abs(padded_dim[1] - current_dim) / 2
    
    # Adjust the mask to be the same as the padded dimensions
    if diff_y != 0:
        mask = mask[..., int(np.floor(diff_y)):-int(np.ceil(diff_y)), :]
    if diff_x != 0:
        mask = mask[..., int(np.floor(diff_x)):-int(np.ceil(diff_x))]
        
    # Calculate the actual size without the padding
    padding = [
        int(abs(face_row["x1"] - face_row["x1_pad"])),
        int(abs(face_row["y1"] - face_row["y1_pad"])),
        int(abs(face_row["x2"] - face_row["x2_pad"])),
        int(abs(face_row["y2"] - face_row["y2_pad"]))
    ]
    
    # Adjust the padding so that it will match the target bounding box
    new_dim = padded_dim[0] - padding[0] - padding[2], padded_dim[1] - padding[1] - padding[3]
    diff_x, diff_y = (target_dim[0] - new_dim[0]) / 2, (target_dim[1] - new_dim[1]) / 2
    
    padding[0] -= int(np.floor(diff_x))
    padding[1] -= int(np.floor(diff_y))
    padding[2] -= int(np.ceil(diff_x))
    padding[3] -= int(np.ceil(diff_y))
    
    mask = F.pad(input=mask, pad=(-padding[0], -padding[2], -padding[1], -padding[3]), mode='constant', value=0)
    
    return mask, True

In [ ]:
def get_features(path):
    torch.autograd.set_detect_anomaly(True)
    
    df = pd.DataFrame() # dataframe storing the dataset
    row = {} #the information/columns for a single row in the dataset is stored here
    grads = []
    bboxes = []
    
    file_basename = os.path.basename(path)
    print(file_basename, end=" ")
    print("<- working on")

    row['path'] = path

    model.eval()
    model.gradient_mode = False

    for yolo_layer in model.yolo_layers:
        yolo_layer.gradient_mode = False

    # Read and transform the image from the path
    data = cv2.imread(path)
    row['source_w'], row['source_h'], _ = data.shape
    data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0)

    with torch.no_grad():
        # Forward pass the data through the model and call non max suppression
        nms, nms_output = non_max_suppression(model(data), 0.5, 0.5) #conf_thres and iou_thres = 0.5

    face_list = []
    if type(nms_output[0]) is not int:
        face_list = nms_output[0]

    data = data.to(device)

    # Set requires_grad attribute of tensor. Important for attack
    data.requires_grad = True

    model.gradient_mode = True
    for yolo_layer in model.yolo_layers:
        yolo_layer.gradient_mode = True

    output = model(data)

    # loop through each of the faces in the image
    for face_index, face_row in enumerate(face_list): #nms_output[0] because the model is designed to take in several images at a time from the dataloader but we are only loading the image one at a time

        row['face_index'] = face_index
        print("Face", face_index)

        row['obj_score'] = face_row[4].item()
        row['class_score'] = face_row[5].item()
        x, y, w, h = face_row[0], face_row[1], face_row[2], face_row[3]

        normal_x, normal_y, normal_w, normal_h = x / 415, y / 415, w / 415, h / 415

        target = torch.tensor([[0.0, 0, normal_x, normal_y, normal_w, normal_h]])

        target = target.to(device)
        loss, loss_components = compute_loss(output, target, model)

        # cropped image with bounding box
        # getting (x1, y1) upper left, (x2, y2) lower right
        x1 = max(int(np.floor((x - w / 2).detach().cpu().numpy())), 0)
        y1 = max(int(np.floor((y - h / 2).detach().cpu().numpy())), 0)
        x2 = min(int(np.ceil((x + w / 2).detach().cpu().numpy())), 415)
        y2 = min(int(np.ceil((y + h / 2).detach().cpu().numpy())), 415)

        row['x1'], row['y1'], row['x2'], row['y2'] = x1, y1, x2, y2
        row['x'], row['y'], row['w'], row['h'] = x, y, w, h
        
        cropped_image = detach_cpu(data)[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
        cropped_image = tensor_to_np_img(cropped_image) #reshape the image to (w/h, h/w, channel)

        # Zero all existing gradients
        model.zero_grad()
        data.grad = None

        # Calculate gradients of model in backward pass
        loss.backward(retain_graph=True) #TODO: Amos - check if this is correct
        
        # Collect datagrad
        data_grad = data.grad.data
        grads.append(clone_detach(data_grad))
        
        bbox = (x1, y1, x2, y2)
        bboxes.append(bbox)
        
        df = pd.concat([df, pd.DataFrame([row])], axis=0, ignore_index=True)
    
    return df, grads, bboxes

In [ ]:
nms_scores = np.linspace(0.5, 1, 10)
conf_scores = np.linspace(0.5, 1, 10)
df = pd.DataFrame()
row = {}

for path in sample_set:
    data = open_img_as_tensor(path)
    feats, grads, bboxes = get_features(path)
    
    #RESTORED_MASK_PATH = os.path.join("../images", path.split("/")[-2] + '_restored_mask')
    #FACES_DF = pd.read_csv(os.path.join("../images", os.path.join(path, path.split("/")[-2] + '_CSV'), file))
    
    for face_index, (data_grad, bbox) in enumerate(zip(grads, bboxes)):
        for nms, conf in itertools.product(nms_scores, conf_scores):
            whole_mask = np.zeros(data.shape)
            whole_mask[..., bbox[1]:bbox[3], bbox[0]:bbox[2]] = 1
            
            print("nms:", nms, "conf:", conf)
            row["nms"] = nms
            row["conf"] = conf

            yn = YoloFace(nms=nms, conf=conf)
            mp = MediaPipe(conf=conf)
            yf = YoloFace(nms=nms, conf=conf)

            yn_min_e_face = fgsm.binary_search(clone_detach(data), clone_detach(data_grad), yn, whole_mask, bbox)
            mp_min_e_face = fgsm.binary_search(clone_detach(data), clone_detach(data_grad), mp, whole_mask, bbox)
            yf_min_e_face = fgsm.binary_search(clone_detach(data), clone_detach(data_grad), yf, whole_mask, bbox)

            print("yn min face:", yn_min_e_face, "mp min face:", mp_min_e_face, "yf min face:", yf_min_e_face)
            row['e_face_yn'], row['e_face_mp'], row['e_face_yf'] = yn_min_e_face, mp_min_e_face, yf_min_e_face
            
            df = pd.concat([df, pd.DataFrame([row])], axis=0, ignore_index=True)
df.to_csv("nmsconf.csv", index=False)

In [ ]:
df

In [ ]:
import matplotlib as plt

nms_group = df[df_results["nms"] >= 0.4].groupby("nms")
plt.plot(nms_group.groups.keys(), nms_group.mean(numeric_only = True).iloc[:, -3:], label=["YuNet", "MediaPipe", "Yolo-face"])
plt.legend()
plt.show()

conf_group = df[df_results["conf"] >= 0.4].groupby("conf")
plt.plot(conf_group.groups.keys(), conf_group.mean(numeric_only = True).iloc[:, -3:], label=["YuNet", "MediaPipe", "Yolo-face"])
plt.legend()
plt.show()

In [ ]:
raise Exception("STOP")

In [ ]:
evil = sample_set.iloc[0, :]["path"]
evil

In [ ]:
last1 = None
last2 = None
row = {}
df = pd.DataFrame()
for path in [evil, evil, evil]:
    print(path)
    row['path'] = path
    REF_SUBSET = sample_set[sample_set['path'] == path]

    model.eval()
    model.gradient_mode = False
    for yolo_layer in model.yolo_layers:
        yolo_layer.gradient_mode = False

    # read and transform the image from the path
    data = cv2.imread(path)  # read the image
    data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
    data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image

    with torch.no_grad():
        # Forward pass the data through the model and call non max suppression
        nms, nms_output = non_max_suppression(model(data), 0.5, 0.5) #conf_thres and iou_thres = 0.5

    face_list = []
    if type(nms_output[0]) is not int:
        face_list = nms_output[0]

    data = data.to(device)
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True

    model.gradient_mode = True
    for yolo_layer in model.yolo_layers:
        yolo_layer.gradient_mode = True

    output = model(data)

    # loop through each of the faces in the image
    for face_index, face_row in enumerate(face_list):
        row['face_index'] = face_index

        if face_index in set(REF_SUBSET['face_index']):
            x, y, w, h = face_row[0], face_row[1], face_row[2], face_row[3]

            factor_x, factor_y, factor_w, factor_h = random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2)
            normal_x, normal_y, normal_w, normal_h = x / 416, y / 416, w / 416, h / 416

            new_x = normal_x * factor_x if random.choice([True, False]) else normal_x / factor_x
            new_y = normal_y * factor_y if random.choice([True, False]) else normal_y / factor_y
            new_w = normal_w * factor_w if random.choice([True, False]) else normal_w / factor_w
            new_h = normal_h * factor_h if random.choice([True, False]) else normal_h / factor_h

            new_x, new_y, new_w, new_h = max(min(1, new_x), 0), max(min(1, new_y), 0), max(min(1, new_w), 0), max(min(1, new_h), 0)

            target = torch.tensor([[0.0, 0, new_x, new_y, new_w, new_h]])
            target = target.to(device)

            loss, loss_components = compute_loss(output, target, model)

            # cropped image with bounding box
            # getting (x1, y1) upper left, (x2, y2) lower right
            x1 = max(int(np.floor((x - w / 2).detach().cpu().numpy())), 0)
            y1 = max(int(np.floor((y - h / 2).detach().cpu().numpy())), 0)
            x2 = min(int(np.ceil((x + w / 2).detach().cpu().numpy())), 415)
            y2 = min(int(np.ceil((y + h / 2).detach().cpu().numpy())), 415)

            cropped_image = detach_cpu(data)[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
            cropped_image = tensor_to_image(cropped_image) #reshape the image to (w/h, h/w, channel)

            # Zero all existing gradients
            model.zero_grad()
            data.grad = None

            # Calculate gradients of model in backward pass
            #loss.backward(retain_graph=True)
            data_grad = autograd.grad(loss, data, torch.ones_like(loss), retain_graph=True)[0]
            #print(data_grad)
            # Collect datagrad
            #data_grad = data.grad.data

            bbox = (x1, y1, x2, y2)

            bbox_mask = np.zeros(data.shape)
            bbox_mask[..., y1:y2, x1:x2] = 1
            
            if (last1 is not None):
                print(torch.any(last1 == data))
                print(torch.any(last2 == data_grad))
            last1 = data
            last2 = data_grad
            
            yn_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, bbox_mask, bbox)
            mp_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, bbox_mask, bbox)
            yf_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, bbox_mask, bbox)

            #print("yn min bbox:", yn_min_e_bbox, "mp min bbox:", mp_min_e_bbox, "yf min bbox:", yf_min_e_bbox)
            row['e_bbox_yn'], row['e_bbox_mp'], row['e_bbox_yf'] = yn_min_e_bbox, mp_min_e_bbox, yf_min_e_bbox

            df = df.append(row, ignore_index=True)
df

In [ ]:
for path in [evil, evil, evil, evil, evil, evil]:
    data = cv2.imread(path)  # read the image
    data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
    data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
    print(minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, bbox_mask, bbox))

In [ ]:
_, yf_face_detector = minE.models.load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")
yf_face_detector.eval()
for i in range(10):
    for path in [evil, evil, evil, evil, evil, evil]:
        image = cv2.imread(path)  # read the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #change to rgb
        #bboxes = minE.detect.detect_image(yf_face_detector, image, conf_thres=0.5, nms_thres=0.5)
        image = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((image, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
        print(yf_face_detector(image))